In [16]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime
import json

In [39]:
folder = Path('eoles') / Path('inputs') / Path('xps')

date = datetime.now().strftime("%Y%m%d")
folder = folder / Path(str(date))

# check if folder is a folder with library pathlib
folder.mkdir(parents=True, exist_ok=True)

base = {
    "no_subsidies": True,
    "subsidies_specified": False,
    "calibration": True,
    "max_iter": 100,
    "health": True,
    "discount_rate": 0.032,
    "carbon_constraint": True,
    "fix_sub_heater": False,
    "fix_sub_insulation": False,
    "list_year": [2020, 2025, 2030, 2035, 2040, 2045],
    "list_trajectory_scc": [0, 250, 350, 500, 650, 775],
    "eoles": {
        "biomass_potential_scenario": "S3",
        "aggregated_potential": True,
        "maximum_capacity_scenario": "N1"
    },
    "subsidy": {
            "proportional_uniform": None,
            "heater": {
                "policy": "subsidy_ad_valorem",
                "proportional": None,
                "cap": None
            },
            "insulation": {
                "policy": "subsidy_ad_valorem",
                "rational_behavior": False,
                "target": None,
                "proportional": None,
                "cap": None
            }
    },
    "scenario_cost_eoles": {
        "fix_capacities": {
            "uiom": 0,
            "CTES": 0
        }
    }
}

folder_base = folder / Path('base.json')
with open(folder_base, "w") as outfile:
    outfile.write(json.dumps(base, indent=4))

name_mapping = {
    'biomass_potential_scenario': 'biogas',
    'maximum_capacity_scenario': 'capacity'
}

dict_supply = {
    'biomass_potential_scenario': ['S3', 'S2'],
    'maximum_capacity_scenario': ['N1nuc', 'N1ren2']
}

dict_demand = {
    'ban': {
            "file": "project/input/policies/current/policies_ambitious.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
        }},
    'ambitious':
        {
            "file": "project/input/policies/current/policies_ambitious.json"
        }
}

In [40]:
config_additional = {
    "name_config": "biogasS3_capacityN1_hcDPE_cc",
    "policies": {
        "file": "project/input/policies/current/policies_ambitious.json"
    },
    "greenfield": False,
    "prices_constant": False,
    "price_feedback": True,
    "biomass_potential_scenario": "S3",
    "aggregated_potential": True,
    "maximum_capacity_scenario": "N1",
    "lifetime_insulation": 5,
    "subsidies_heater": None,
    "subsidies_insulation": None,
    "optim_eoles": True,
    "carbon_budget": "carbon_budget_tight_convex",
    "district_heating_potential": None,
    "dict_configs": None,
    "cap_MWh": 1000,
    "cap_tCO2": 2000,
    "demand_scenario": "Reference",
    "method_health_cost": "epc"
}

list_configs = []
for supply, scenario in dict_supply.items():
    for s in scenario:
        for demand, policy in dict_demand.items():
            new_config = config_additional.copy()
            new_config[supply] = s
            new_config['policies'] = policy
            new_name = name_mapping[supply] + s
            name_config = '_'.join([new_name if name_mapping[supply] in x else x for x in new_config['name_config'].split('_')])
            name_config = name_config + f'_policy{demand}'
            new_config['name_config'] = name_config
            list_configs.append(new_config)
            folder_additional = folder / Path(name_config + '.json')
            with open(folder_additional, "w") as outfile:
                outfile.write(json.dumps(new_config, indent=4))